# 📊 Parcial – Análisis de Redes Sociales (Primer Corte)

## 🎯 Objetivo
Evaluar tu dominio de **filtros** y **agrupaciones** en Python (sin pivots ni gráficos) y tu capacidad de **análisis de negocio** en métricas de marketing: **CAC, LTV, churn rate, funnel, unit economics (LTV/CAC)**.

---
## 🧩 Contexto de Negocio (Caso Real)
**StartUp SaaS en crecimiento.** Vende planes **Basic, Pro, Enterprise** en **LatAm, North America, Europe, APAC**. Los canales de adquisición son: **meta_ads, google_ads, organic_search, outbound_sales**. Los usuarios se registran por **web** o **mobile_app**.

La dirección quiere decidir **dónde invertir el presupuesto del próximo trimestre**. Te piden:

1) Identificar **canales más rentables** (relación **LTV/CAC** y churn).  
2) Detectar **regiones** con mayor **potencial de crecimiento** y/o **riesgo**.  
3) Evaluar el desempeño por **tier** (Basic, Pro, Enterprise).  
4) Recomendar **asignación de presupuesto** por **canal + tier**.

---
## 📂 Instrucciones
- Trabaja únicamente con **filtros** y **agrupaciones**. **No uses** tablas dinámicas (pivot), merges ni gráficos.
- Archivo a usar: `clientes_marketing.csv`.
- Escribe **código + interpretación breve** para cada inciso.
- Si el enunciado dice **“Agrupa”**, usa `groupby`. Si dice **“Filtra”**, usa filtrado con máscaras booleanas. Si dice **“Ambas”**, realiza primero el **filtro** y luego la **agrupación**.

---


## Descargar Tabla (Correr una vez)

In [1]:
!wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv

--2025-09-17 23:13:36--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv [following]
--2025-09-17 23:13:37--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/cac_ltv_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661398 (646K) [text/plain]
Saving to: ‘cac_ltv_model.csv’

cac_ltv_model.csv   100%[===================>] 645.90K  2.64MB

## Importar Datos

In [107]:
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# 1) Cargar dataset
data = pd.read_csv('cac_ltv_model.csv')


### ✅ Variables relevantes (recordatorio)
- `acquisition_channel`, `signup_source`, `region`, `customer_tier`
- `plan_price`, `discount_rate`, `arpu`, `gross_margin`, `churn_rate`
- `contract_length_months`, `marketing_spend`

### 🧮 Fórmulas de negocio (a usar en el examen)
- **CAC por canal**:  
    $$ CAC = \frac{\text{Total Marketing Spend}}{\text{Clientes adquiridos}} $$
  
- **LTV por observación** (aprox.):  
   $$ LTV = ARPU \times contract\_length\_months \times gross\_margin \times (1 - churn\_rate) $$

- **Unit economics**:  
  $$ LTV/CAC   $$(ideal \> 3 como regla general de SaaS)

---



### Parte Exploratoria: Cree una muestra de los primeros 5 elementos de esta tabla. ¿Cuantas observaciones tienes?

In [108]:
data.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27


## 🔎 Parte A – Filtros

hint: Crea una tabla con el filtro y luego agrupa la variable que te piden.

1. **Filtra** los clientes del canal **`meta_ads`**. Calcula el **ARPU promedio** de este subconjunto.  



In [109]:
meta_ads = data[data['acquisition_channel'] == 'meta_ads']
meta_ads['arpu'].mean()

np.float64(168.78858272162617)

2. **Filtra** clientes de **LatAm** con **`churn_rate > 0.05`**. ¿Cuántos son? ¿Cuál es su **ARPU promedio**?  

In [110]:
LatAm = data[(data['region'] == 'LatAm') & (data['churn_rate'] > 0.05)]
LatAm ['arpu'].mean()

np.float64(174.78453658536586)

3. **Filtra** clientes **Enterprise** con `contract_length_months > 6`. ¿Cuál es su **gross_margin promedio**?

In [111]:
Enterprise = data[(data['customer_tier'] == 'Enterprise') & (data['contract_length_months'] > 6)]
Enterprise['gross_margin'].mean()

np.float64(0.8248397435897435)

## 📊 Parte B – Agrupaciones (SOLO agrupar)
4. **Agrupa** por `acquisition_channel` y calcula **ARPU promedio**. Ordena de mayor a menor.  


In [112]:
data.groupby("acquisition_channel")["plan_price"].mean().sort_values(ascending=False)

,plan_price
acquisition_channel,
outbound_sales,192.84
google_ads,191.44
organic_search,189.20
meta_ads,187.30


5. **Agrupa** por `region` y calcula **churn_rate promedio**. Identifica la región con mayor churn.  



In [113]:
data.groupby("region")["churn_rate"].mean().sort_values(ascending=False)

,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


6. **Agrupa** por `customer_tier` y calcula **marketing_spend total**. ¿Cuál tier consume más presupuesto?

In [114]:
data.groupby("customer_tier")["marketing_spend"].sum().sort_values(ascending=False)

,marketing_spend
customer_tier,
Pro,"589,265.04"
Basic,"584,419.11"
Enterprise,"584,348.87"


## 🔀 Parte C – Filtro **y** Agrupación (Ambas)
7. **Filtra** solo registros de **`signup_source = 'web'`** y luego **agrupa** por `acquisition_channel` para obtener el **churn_rate promedio**.  


In [115]:
signup_source = data[(data['signup_source'] == 'web')]
signup_source.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27
6,2023,1,Jan-23,1007,google_ads,web,North America,Basic,78.34,0.00,73.19,0.83,0.09,1,348.95


In [116]:
signup_source.groupby("acquisition_channel")["churn_rate"].mean().sort_values(ascending=False)

,churn_rate
acquisition_channel,
google_ads,0.05
meta_ads,0.05
organic_search,0.05
outbound_sales,0.05


8. **Filtra** solo **`mobile_app`** y luego **agrupa** por `region` para calcular **ARPU promedio**.  


In [117]:
mobile_app = data[(data['signup_source'] == 'mobile_app')]
mobile_app.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40
5,2023,1,Jan-23,1006,organic_search,mobile_app,APAC,Pro,138.52,0.10,133.17,0.80,0.08,1,56.92
12,2023,1,Jan-23,1013,outbound_sales,mobile_app,APAC,Pro,111.21,0.00,105.14,0.77,0.07,1,219.08
15,2023,1,Jan-23,1016,meta_ads,mobile_app,Europe,Enterprise,282.59,0.00,293.98,0.85,0.07,1,355.55
19,2023,1,Jan-23,1020,meta_ads,mobile_app,North America,Pro,197.22,0.00,181.95,0.76,0.03,12,363.98


In [118]:
mobile_app.groupby("region")["arpu"].mean().sort_values(ascending=False)

,arpu
region,
North America,174.00
LatAm,173.20
Africa,172.60
Middle East,171.71
APAC,169.09
Europe,165.89


9. **Filtra** a clientes **`Pro`** y **agrupa** por `acquisition_channel` para calcular **marketing_spend total**.


In [119]:
Pro = data[(data['customer_tier'] == 'Pro')]
Pro.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10
5,2023,1,Jan-23,1006,organic_search,mobile_app,APAC,Pro,138.52,0.10,133.17,0.80,0.08,1,56.92
8,2023,1,Jan-23,1009,outbound_sales,partner,APAC,Pro,108.54,0.20,81.91,0.80,0.02,12,223.23
9,2023,1,Jan-23,1010,google_ads,partner,North America,Pro,183.89,0.00,187.27,0.87,0.08,1,391.30


In [120]:
Pro.groupby("acquisition_channel")["marketing_spend"].sum().sort_values(ascending=False)

,marketing_spend
acquisition_channel,
google_ads,"226,721.68"
meta_ads,"211,517.60"
outbound_sales,"121,177.04"
organic_search,"29,848.72"


## 📈 Parte D – Métricas de negocio (CAC, LTV, LTV/CAC)
10. **CAC por canal (Agrupa)**: calcula el CAC de cada `acquisition_channel` como:  
   `CAC = marketing_spend_total_del_canal / #clientes_del_canal`  


In [121]:
data.groupby("acquisition_channel")["marketing_spend"].sum() / data.groupby("acquisition_channel")["customer_id"].count()

,0
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


11. **CAC por tier (Agrupa)**: calcula el CAC de cada `customer_tier` como:  
   `CAC = marketing_spend_total_del_tier / #clientes_del_canal`  


In [122]:
data.groupby("customer_tier")["marketing_spend"].sum() / data.groupby("customer_tier")["customer_id"].count()

,0
customer_tier,
Basic,249.75
Enterprise,249.19
Pro,248.43


12. **LTV por tier (Ambas)**: crea una columna `ltv_individual` con la fórmula dada.


In [133]:
data['ltv_individual'] = data['arpu'] * data['contract_length_months'] * data['gross_margin'] * (1 - data["churn_rate"])
data.head()

,year,month,date,customer_id,acquisition_channel,signup_source,region,customer_tier,plan_price,discount_rate,arpu,gross_margin,churn_rate,contract_length_months,marketing_spend,ltv_individual
0,2023,1,Jan-23,1001,outbound_sales,web,North America,Basic,78.84,0.10,63.63,0.76,0.02,12,212.48,567.54
1,2023,1,Jan-23,1002,meta_ads,web,LatAm,Pro,120.23,0.00,124.47,0.84,0.05,1,352.60,99.20
2,2023,1,Jan-23,1003,organic_search,mobile_app,North America,Enterprise,335.82,0.20,278.20,0.81,0.02,12,55.40,"2,651.91"
3,2023,1,Jan-23,1004,organic_search,web,Europe,Pro,193.89,0.10,190.65,0.80,0.08,1,49.10,139.68
4,2023,1,Jan-23,1005,organic_search,web,APAC,Enterprise,471.80,0.10,445.32,0.80,0.08,1,45.27,328.72


12. A. Luego **agrupa** `ltv_individual` por
`acquisition_channel` para obtener el **LTV promedio**.  

In [134]:
data.groupby("acquisition_channel")["ltv_individual"].mean()

,ltv_individual
acquisition_channel,
google_ads,710.25
meta_ads,729.05
organic_search,765.13
outbound_sales,751.57


12. B. Luego **agrupa** `ltv_individual`por `customer_tier` para obtener el **LTV promedio**.





In [135]:
data.groupby("customer_tier")["ltv_individual"].mean()

,ltv_individual
customer_tier,
Basic,291.77
Enterprise,"1,349.02"
Pro,577.63


13. **Unit economics (Ambas)**: combina tus resultados para comparar **LTV promedio por tier** contra **CAC por canal** y comenta **qué combinaciones canal + tier** lucen más saludables (busca **LTV/CAC > 3**).


In [149]:
data.groupby("customer_tier")["ltv_individual"].mean()
data.groupby("customer_tier")["marketing_spend"].sum() / data.groupby("customer_tier")["customer_id"].count()
LTV/CAC > 3

,0
customer_tier,
Basic,False
Enterprise,True
Pro,False


El mas saludable es el nivel de cliente: empresario

## 🧠 Parte E – Análisis crítico (respuesta abierta)
13. **CMO por un día**: con tus resultados, ¿en qué **canal(es)** invertirías más el próximo trimestre? ¿Por qué? (Cita **CAC** y **churn**).  
14. ¿Qué **región** ves con mayor riesgo? ¿Qué hipótesis explicarían su **churn**?  
15. Identifica **dos combinaciones canal + tier** con mejor **LTV/CAC** y propón **dos acciones** concretas de optimización (p. ej., creatividades, audiencias, pricing, retención).

---
✍️ **Nota**: Mantente disciplinado con el enunciado (**Filtra**, **Agrupa**, **Ambas**). La evaluación pondera **correctitud técnica** y **calidad de interpretación de negocio**.


In [144]:
#13
data.groupby("acquisition_channel")["marketing_spend"].sum() / data.groupby("acquisition_channel")["customer_id"].count()

,0
acquisition_channel,
google_ads,401.02
meta_ads,349.27
organic_search,49.95
outbound_sales,200.22


# Respuesta 13
El canal en el que se debe invertir mas en el siguiente trimestre es en la busqueda organica, puesto que segun los datos ha tenido un costo de 200.22, el menor a comparacion de los demas canales, adicional a esto todos los canales tienen la misma tasa de abandono (0.05), lo que quiere decir que todos pueden ser efectivos potencializandolos de igual forma

In [145]:
#14
data.groupby("region")["churn_rate"].mean().sort_values(ascending=False)

,churn_rate
region,
Middle East,0.05
LatAm,0.05
North America,0.05
APAC,0.05
Europe,0.05
Africa,0.05


# Respuesta 14
Todas las regiones tienen la misma tasa de abandono, por lo que todas tienen un mismo nivel de riesgo, esto se puede explicar como un problema no directamente relacionado a lo local o variables dependientes al entorno, si no mas bien un problema interno de la organizacion en la rgeion, por ejemplo el modelo de negocio, la propuesta de valor, problemas en la retencion de clientes etc


In [150]:
#15 LTV promedio tier
data.groupby("customer_tier")["ltv_individual"].mean()

,ltv_individual
customer_tier,
Basic,291.77
Enterprise,"1,349.02"
Pro,577.63


# Respuesta 15
Organic search

Acción 1: invertir en SEO para empresas con contenidos como estudios de caso, blogs y/o artículos basados en investigacion para atraer clientes enterprise

Acción 2: optimizar onboarding y soporte premium para garantizar que esos clientes permanezcan más tiempo gracias al valor agregado

Outbound Sales

Acción 1: generar nuevos pitches segmentados y espacios personalizados para clientes enterprise.

Acción 2: ajustar el precio de servicio / producto con beneficios exclusivos para Enterprise que incentive acuerdos comerciales o una relacion mas duradera por costo/beneficio
